In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F

import torchmetrics

import onnx
import onnxruntime.quantization
import onnxruntime
from onnxruntime.quantization import quantize_qat, quantize_static, QuantType

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# WANDB LIBRARY
### IMPORT WANDB AND LOGIN
Note you may have to login using your API key


In [3]:
import wandb
%env "WANDB_NOTEBOOK_NAME" "demo_wine_wandb_test"
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


env: "WANDB_NOTEBOOK_NAME"="demo_wine_wandb_test"


wandb: Currently logged in as: markgich (use `wandb login --relogin` to force relogin)


True

### Import Dataset

In [4]:
df = pd.read_csv("./data/wine_data.csv")

In [5]:
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [6]:
df.shape

(178, 14)

In [7]:
df.describe()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,0.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,0.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,0.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,1.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,2.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,2.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


# DATA WRANGLING
### Check for Nulls and Duplicates

In [8]:
df.isna().sum()

Class                           0
Alcohol                         0
Malic acid                      0
Ash                             0
Alcalinity of ash               0
Magnesium                       0
Total phenols                   0
Flavanoids                      0
Nonflavanoid phenols            0
Proanthocyanins                 0
Color intensity                 0
Hue                             0
OD280/OD315 of diluted wines    0
Proline                         0
dtype: int64

In [9]:
df.duplicated().sum()

0

# MACHINE LEARNING
### ML PREP

In [10]:
# Encode target labels with value between 0 and n_classes-1.
# Import Metrics for use with evaluation

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, confusion_matrix


In [11]:
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df.sample(10)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
48,0,14.10,2.02,2.40,18.8,103,2.75,2.92,0.32,2.38,6.20,1.07,2.75,1060
62,1,13.67,1.25,1.92,18.0,94,2.10,1.79,0.32,0.73,3.80,1.23,2.46,630
152,2,13.11,1.90,2.75,25.5,116,2.20,1.28,0.26,1.56,7.10,0.61,1.33,425
77,1,11.84,2.89,2.23,18.0,112,1.72,1.32,0.43,0.95,2.65,0.96,2.52,500
137,2,12.53,5.51,2.64,25.0,96,1.79,0.60,0.63,1.10,5.00,0.82,1.69,515
134,2,12.51,1.24,2.25,17.5,85,2.00,0.58,0.60,1.25,5.45,0.75,1.51,650
105,1,12.42,2.55,2.27,22.0,90,1.68,1.84,0.66,1.42,2.70,0.86,3.30,315
169,2,13.40,4.60,2.86,25.0,112,1.98,0.96,0.27,1.11,8.50,0.67,1.92,630
44,0,13.05,1.77,2.10,17.0,107,3.00,3.00,0.28,2.03,5.04,0.88,3.35,885
120,1,11.45,2.40,2.42,20.0,96,2.90,2.79,0.32,1.83,3.25,0.80,3.39,625


In [12]:
df['Class'].unique()

array([0, 1, 2], dtype=int64)

### SEPARATE FEATURES AND TARGET

In [13]:
# set the feature variables

df_features = df.drop('Class', axis=1)
df_features.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [14]:
# Set the target variable

df_target = df[['Class']]
df_target.head()

,Class
0,0
1,0
2,0
3,0
4,0


In [15]:
# Split dataset in train and test with a ratio of 70-30

from sklearn.model_selection import train_test_split

In [16]:
X_train, x_test, Y_train, y_test = train_test_split(df_features, 
                                                    df_target,
                                                    test_size=0.3,
                                                     random_state=42)

In [17]:
X_train.shape, x_test.shape,

((124, 13), (54, 13))

In [18]:
Y_train.shape, y_test.shape

((124, 1), (54, 1))

### Convert data to Tensors for Pytorch

In [19]:
Xtrain = torch.from_numpy(X_train.values).float()
Xtest = torch.from_numpy(x_test.values).float()
print(Xtrain.shape, Xtest.shape)

torch.Size([124, 13]) torch.Size([54, 13])


In [20]:
print(Xtrain.dtype, Xtest.dtype)

torch.float32 torch.float32


We have successfully converted our  X_data into torch tensors of float32 datatype

In [21]:
# Reshape tensor to 1D

Ytrain = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest = torch.from_numpy(y_test.values).view(1, -1)[0]
print(Ytrain.shape, Ytest.shape)
print(Ytrain.dtype, Ytest.dtype)

torch.Size([124]) torch.Size([54])
torch.int64 torch.int64


We use the **view()** to reshape the tensor.<br>
The loss function doesn't support multi-target and therefore, we should use a 1D Tensor of 1 row containing the labels.<br>
We have successfully converted our y_data

In [22]:
print(Ytrain.dtype, Ytest.dtype)

torch.int64 torch.int64


## PyTorch
### We create a classifier and define our neural network for our model

### Hyperparameters

In [23]:
input_size = 13
output_size = 3
hidden_size = 100

In [24]:
config = dict(
                dataset = "wine dataset",
                architecture = 'Linear', 
                learning_rate = 0.01,
                loss = nn.NLLLoss(),
                optimizer = "adam",
)


In [25]:
for k,v in config.items():
    print(k, v)

dataset wine dataset
architecture Linear
learning_rate 0.01
loss NLLLoss()
optimizer adam


### Define the neural network


class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(config.get("input_size"), config.get("hidden_size"))
        self.fc2 = nn.Linear(config.get("input_size"), config.get("hidden_size"))
        self.fc3 = nn.Linear(config.get("input_size"), config.get("output_size"))

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [26]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

        self.dropout = nn.Dropout(0.25)


    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = self.dropout(X)
        X = torch.sigmoid(self.fc2(X))
        X = self.dropout(X)
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [27]:
# instantiate model
model = Net()
# move model to gpu
#model.to(device)
# preview our model
model

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

### Define Optimizer and Loss Function

In [28]:
import torch.optim as optim

In [29]:
#optimizer = optim.Adam(model.parameters(), lr=config.get("learning_rate"))
loss_fn = config.get("loss")

In [30]:
# Define the optimizer

if config.get("optimizer")=='sgd':
    optimizer = optim.SGD(model.parameters(),lr=config.get("learning_rate"), momentum=0.9)
elif config.get("optimizer")=='adam':
    optimizer = optim.Adam(model.parameters(),lr=config.get("learning_rate"))

In [31]:
# Define the Loss

if config.get("loss") == "NLLLoss":
    loss_fn = nn.NLLLoss()
elif config.get("loss") == "CrossEntropyLoss":
    loss_fn = nn.CrossEntropyLoss()

# TRAIN AND LOG THE MODEL
### Train the Model for 1000 Epochs
### Log the model Parameters
### Export Model, Import and Set to Eval
### Log Metrics on Model.Eval
### Convert and Export to ONNX

# TRAIN THE MODEL


epochs = 1000
with wandb.init(project="demo_wandb_test", config = config):
    wandb.watch(model, criterion=None, log="all", log_freq=10)
    
    for epoch in range(epochs):
        # zero the gradients
        optimizer.zero_grad()
        # train the model
        Ypred = model(Xtrain)
        # compute the accuract
        acc = torchmetrics.functional.accuracy(Ypred, Ytrain)
        # compute the loss
        loss = loss_fn(Ypred, Ytrain)
        # update the model weights
        loss.backward()
        # optimize the learning parameters and step forward
        optimizer.step()
        # log the metrics
        wandb.log({'Epoch': epoch, "Loss": loss.item(), "Accuracy": acc})


    # SAVE MODEL STATE DICT TO DISK

    wandb.save(torch.save(model.state_dict(), "./models/home_state_dict.pt"))

    # LOAD MODEL FROM DISK and EVALUATE

    new_model =  Net()
    new_model.load_state_dict(torch.load("./models/home_state_dict.pt"))
    new_model.eval()

    # SET THE PREDICTIONS

    predict = new_model(Xtest)
    _, predict_y = torch.max(predict, 1)


    # VISUALIZE CONFUSION MATRIX
    wandb.sklearn.plot_confusion_matrix(Ytest, predict_y, labels = [0,1,2])

    # Print Metrics

    wandb.log({"accuracy_score" : accuracy_score(Ytest, predict_y),
    "precision_score" : precision_score(Ytest, predict_y, average='weighted'),
    "recall_score": recall_score(Ytest, predict_y, average="weighted"),
    
    })
    
    torch.onnx.export(model = model,args =  (Xtrain), f = "./models/home_state_test.onnx", input_names=['input'], output_names = ['output'],
    verbose=True, do_constant_folding=True, opset_version=11)
wandb.finish()


# COMPUTE METRICS ON EACH RUN
## TRAIN AND EVALUATE LOOP

In [32]:
# TRAIN THE MODEL
def train():
    epochs = 1000
    accuracy = 0
    with wandb.init(project="demo_wandb_test", config = config):
        wandb.watch(model, criterion=None, log="all", log_freq=10)
        # train
        for epoch in range(epochs):
            # zero the gradients
            optimizer.zero_grad()
            # train the model and get an output
            Ypred = model(Xtrain)
            # compute the accuracy
            acc = torchmetrics.functional.accuracy(Ypred, Ytrain)
            accuracy += int(sum(Ypred == Ytest))
            # compute the loss
            loss = loss_fn(Ypred, Ytrain)
            # update the model weights
            loss.backward()
            # optimize the learning parameters and step forward
            optimizer.step()
            # log the metrics
            wandb.log({'Epoch': epoch, "Train Loss": loss.item(), "Torch Accuracy": acc, "Train Accuracy":accuracy})


        # SAVE MODEL STATE DICT TO DISK

        wandb.save(torch.save(model.state_dict(), "./models/home_state_dict.pt"))

        # EVALUATE
        # LOAD MODEL FROM DISK and EVALUATE


        new_model =  Net()
        new_model.load_state_dict(torch.load("./models/home_state_dict.pt"))
        new_model.eval()

        # SET THE PREDICTIONS

        predict = new_model(Xtest)
        _, predict_y = torch.max(predict, 1)


        # VISUALIZE CONFUSION MATRIX
        wandb.sklearn.plot_confusion_matrix(Ytest, predict_y, labels = [0,1,2])

        # Print Metrics

        wandb.log({"accuracy_score" : accuracy_score(Ytest, predict_y),
        "precision_score" : precision_score(Ytest, predict_y, average='weighted'),
        "recall_score": recall_score(Ytest, predict_y, average="weighted"),
        
        })
        
        torch.onnx.export(model = model,args =  (Xtrain), f = "./models/home_state_test.onnx", input_names=['input'], output_names = ['output'],
        verbose=True, do_constant_folding=True, opset_version=11)
    wandb.finish()


In [33]:
train()

RuntimeError: The size of tensor a (3) must match the size of tensor b (54) at non-singleton dimension 1

# OPTIMIZE NETWORK WITH SWEEPS
### WANDB SWEEPS

In [ ]:
# Replace agent with own agent line generated from project
!wandb agent markgich/demo_wandb_test/izsvluxh

In [ ]:
sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [100, 500, 1000]
        },
        
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        'fc_layer_size':{
            'values':[128,256,512]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'loss':{
            'values':["NLLLoss", "CategoricalCrossEntropy"]
        }
    }
}
sweep_id = wandb.sweep(sweep_config)
#wandb.agent(sweep_id=sweep_id, project="demo_wandb_test", )

# ADD HYPERPARAMETER TUNING SWEEPS

# DEMO OF IT
def train():
    with wandb.init() as run:
        config = wandb.config
        model = Model(config)
        for epoch in range(config["epochs"]):
            loss = model.fit()  # your model training code here
            wandb.log({"loss": loss, "epoch": epoch})

count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

In [ ]:

def train():
    # TRAIN THE MODEL
    epochs = 1000
    with wandb.init(project="demo_wandb_test", config = config):
        #wandb.watch(model, criterion=None, log="gradients", log_freq=10)
        
        for epoch in range(epochs):
            # zero the gradients
            optimizer.zero_grad()
            # train the model
            Ypred = model(Xtrain)
            # compute the accuract
            acc = torchmetrics.functional.accuracy(Ypred, Ytrain)
            # compute the loss
            loss = loss_fn(Ypred, Ytrain)
            # update the model weights
            loss.backward()
            # optimize the learning parameters and step forward
            optimizer.step()
            # log the metrics
            wandb.log({'Epoch': epoch, "Loss": loss.item(), "Accuracy": acc})


        # SAVE MODEL STATE DICT TO DISK

        wandb.save(torch.save(model.state_dict(), "./models/home_state_dict.pt"))

        # LOAD MODEL FROM DISK and EVALUATE

        new_model =  Net()
        new_model.load_state_dict(torch.load("./models/home_state_dict.pt"))
        new_model.eval()

        # SET THE PREDICTIONS

        predict = new_model(Xtest)
        _, predict_y = torch.max(predict, 1)


        # VISUALIZE CONFUSION MATRIX
        wandb.sklearn.plot_confusion_matrix(Ytest, predict_y, labels = [0,1,2])

        # Print Metrics

        wandb.log({"accuracy_score" : accuracy_score(Ytest, predict_y),
        "precision_score" : precision_score(Ytest, predict_y, average='weighted'),
        "recall_score": recall_score(Ytest, predict_y, average="weighted"),
        
        })
        
        torch.onnx.export(model = model,args =  (Xtrain), f = "./models/home_state_test.onnx", input_names=['input'], output_names = ['output'],
        verbose=True, do_constant_folding=True, opset_version=11)
    wandb.finish()

count = 5 # number of runs to execute
wandb.agent(sweep_id = sweep_id, project="demo_wandb_test", function=train(), count=count)
#wandb.agent(sweep_id=sweep_id, project="demo_wandb_test", )